# Google Gemini using Langchain

In [1]:
import sys, os
# import google.generativeai as genai
sys.path.append(os.path.abspath(os.path.join('..', 'secret')))
from secret_info import google_genai_api

GOGGLE_API_KEY = google_genai_api
# genai.configure(api_key = GOGGLE_API_KEY)

## LLMs wrappers

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7, safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },)
chat.invoke("explain how a car works in one sentence").content

"A car's engine converts the chemical energy in gasoline into mechanical energy, which is transmitted through a transmission to the wheels, causing them to rotate and propel the car forward."

In [3]:
# Same as above
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)
llm.invoke("Why are corals important to ocean life and our society")

'**Importance of Corals to Ocean Life:**\n\n* **Habitat and Shelter:** Corals provide a complex and intricate habitat for a vast array of marine organisms, including fish, invertebrates, and algae. They offer shelter, protection from predators, and feeding grounds.\n* **Biodiversity Hotspot:** Coral reefs are known as the "rainforests of the sea" due to their incredible biodiversity. They support over 25% of all marine species, including many threatened and endangered ones.\n* **Food Source:** Corals and the organisms they support provide a vital food source for various marine animals, including fish, turtles, and seabirds.\n* **Oxygen Production:** Corals, through photosynthesis, contribute significantly to oxygen production in the oceans. They play a crucial role in maintaining a healthy marine ecosystem.\n* **Water Filtration:** Corals help filter and purify seawater, removing pollutants and sediment. They improve water quality and clarity, supporting the growth of other marine orga

## CHAT MODEL

#### Coding

In [5]:
from langchain.schema import HumanMessage, SystemMessage

chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script to calculate the fibonacci sequence ")
]
response = chat.invoke(input=messages)
print(response.content,end='\n')

```python
def fibonacci(n):
    """Calculates the nth fibonacci number.

    Args:
        n: The index of the fibonacci number to calculate.

    Returns:
        The nth fibonacci number.
    """

    if n < 2:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)


if __name__ == "__main__":
    n = int(input("Enter the index of the fibonacci number to calculate: "))
    print(f"The {n}th fibonacci number is {fibonacci(n)}")
```


## PROMPTS

### Single Prompt Interaction 

In [8]:
# Single promt interaction: 
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import SimpleSequentialChain

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY)
# First prompt
template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in 200 words
"""
prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)
chain = prompt | llm
chain.invoke({'concept': 'training'})

"Training in deep learning involves fine-tuning the model's parameters to optimize its performance on a given dataset. It is an iterative process where the model learns from the data and adjusts its internal structure to improve its accuracy.\n\nDuring training, the model is presented with a set of labeled data, where the input data is paired with the corresponding output or target. The model then propagates the input data through its layers, generating an output prediction. This prediction is compared to the true label, and the difference, known as the loss, is calculated.\n\nThe model's parameters are then updated using a backpropagation algorithm, which calculates the gradients of the loss function with respect to the parameters. These gradients provide information about how the parameters affect the loss, allowing the model to adjust them in a way that minimizes the loss.\n\nThe training process continues through multiple epochs, where each epoch involves passing the entire dataset

### Multi Prompt Interactions 

This portion uses LCEL as SimpleSequentialChain and LLChains are now deprecated, and generate an error. See: 

https://stackoverflow.com/questions/78508462/keyerror-in-simplesequentialchain/78562683#78562683

https://github.com/langchain-ai/langchain/discussions/20137

In [9]:
# LCEL
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY)

template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in 200 words
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

second_prompt = PromptTemplate(
    input_variables=["structure"],
    template="Turn the concept description of {structure} and explain it to me like I'm five in 500 words",
)
initial_chain = prompt | llm | StrOutputParser()
review_chain = second_prompt | llm | StrOutputParser()

chain = ({"structure" : initial_chain} 
         | RunnablePassthrough.assign(review=review_chain)
        )
result = chain.invoke({"concept": 'neural network'})
result

{'structure': 'A neural network is a type of machine learning algorithm inspired by the human brain. It consists of interconnected layers of nodes, called neurons, which process data and transmit it to the next layer. Each neuron takes in a weighted sum of its inputs and applies an activation function to produce an output.\n\nNeural networks are trained on large datasets using a process called backpropagation. During training, the network learns to adjust its weights and biases to minimize the error between its predictions and the true labels. Once trained, neural networks can be used for tasks such as image recognition, natural language processing, and speech recognition.',
 'review': "Imagine your brain as a giant playground filled with tiny workers called neurons. These neurons are like little computers that can do simple math and talk to each other.\n\nNow, let's say you want to teach your brain to recognize cats. You show it a bunch of pictures of cats and non-cats, and it tries t

In [10]:
# This example generates as error 
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import SimpleSequentialChain

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY)
# First prompt
template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in 200 words
"""
prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)
chain = prompt | llm

# Define a second prompt 
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = second_prompt | llm

overall_chain = SimpleSequentialChain(chains=[chain, chain_two])
# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)

KeyError: 'chains'

## Pandas DATAFRAME

https://python.langchain.com/v0.1/docs/integrations/toolkits/pandas/

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)